In [1]:
import cv2
import numpy as np
import os
import random
import torch
import config

from model_structure import UNet
from tqdm import tqdm

In [2]:
# Load the PyTorch model
model = UNet()
model.load_state_dict(torch.load("./model_for_vasc.pth", map_location="cpu"))

# Set the model to evaluation mode
model.eval()

# Directory paths for input and output data
input_dir = "./preparations/data/indata"
output_dir = "./preparations/data/outdata"

# Get a list of image files in the input directory
image_files = os.listdir(input_dir)

In [15]:
while True:
    # Choose a random pair of input and output files
    random_image_file = random.choice(image_files)

    # Construct the full file paths
    input_image_path = os.path.join(input_dir, random_image_file)
    output_label_path = os.path.join(output_dir, random_image_file)
    
    label_image = cv2.imread(output_label_path, cv2.IMREAD_GRAYSCALE)
    shape = label_image.shape
    random_x, random_y = random.randint(0, shape[0] - config.WIDTH - 1), random.randint(0, shape[1] - config.HEIGHT - 1)
    label_image = label_image[random_y : random_y + config.HEIGHT, random_x : random_x + config.WIDTH]
    if cv2.countNonZero(label_image) == 0:
        continue
    input_image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)[random_y : random_y + config.HEIGHT, random_x : random_x + config.WIDTH]
    break

# Perform inference using the model
with torch.no_grad():
    input_tensor = torch.from_numpy(input_image / 255).unsqueeze(0).float()
    output = model(input_tensor)
# Convert the output tensor to a NumPy array
output_np = output.numpy()
# Convert the output prediction to binary format and multiply by 255
print((np.unique(output_np).size))
binary_output = (output_np >= 0.5).astype(np.uint8)[0] * 255
# Display the input image, label, and binary prediction using OpenCV
cv2.imshow("Input Image", cv2.resize(input_image, (512, 512)))
cv2.imshow("Label Image", cv2.resize(label_image, (512, 512)))
print(cv2.resize(binary_output, (512, 512)).shape)
cv2.imshow("Model Prediction (Binary)", cv2.resize(binary_output, (512, 512)))
cv2.imshow("Correctness Image", cv2.resize((np.multiply(binary_output, label_image) + np.multiply((255 - binary_output), (255 - label_image))) * 255, (512, 512)))

print(np.sum(np.multiply(binary_output, label_image) + np.multiply((255 - binary_output), (255 - label_image))) / 64 / 64)

# Wait for a key press and then close the OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()

4095
(512, 512)
0.987548828125


In [17]:
def get_prediction(img, margin):
    if margin != 0:
        raise ValueError
    with torch.no_grad():
        input_tensor = torch.from_numpy(img / 255).unsqueeze(0).float()
        output = model(input_tensor)
    output_np = output.numpy()
    return output_np

In [39]:
input_dir = "./preparations/data/indata"
output_dir = "./preparations/data/outdata"

margin = 0
step = 16
# Get a list of image files in the input directory
image_files = os.listdir(input_dir)

random_image_file = random.choice(image_files)
input_image_path = os.path.join(input_dir, random_image_file)
output_label_path = os.path.join(output_dir, random_image_file)

input_image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)
label_image = cv2.imread(output_label_path, cv2.IMREAD_GRAYSCALE)

shape = label_image.shape

input_image = input_image[shape[0] % step:, shape[1]%step:]
label_image = label_image[shape[0] % step:, shape[1]%step:]

shape = label_image.shape
count_times = np.zeros(shape)
total_times = np.zeros(shape)

arr1 = []
arr2 = []
for i in tqdm(range(0, shape[0] - config.HEIGHT, step), "Processing"):
    for j in range(0, shape[1] - config.WIDTH, step):
        slice = input_image[i : i + config.HEIGHT, j : j + config.WIDTH]
        # print(i, j, slice.shape)
        array_tmp = get_prediction(slice, margin)[0]
        # print(array_tmp.shape)
        count_times[i : (i + config.HEIGHT), j : (j + config.WIDTH)] += (array_tmp >= 0.5)
        
        total_times[i : (i + config.HEIGHT), j : (j + config.WIDTH)] += 1

        

unique = np.unique(count_times)
confidence = (count_times + 0.000000001) / (total_times + 0.001)
binary_output = (np.floor(confidence * 255))

cv2.imshow("Input Image", cv2.resize(input_image, (512, 512)))
cv2.imshow("Label Image", cv2.resize(label_image, (512, 512)))
cv2.imshow("Model Prediction (Binary)", cv2.resize(binary_output, (512, 512)))
cv2.imshow("Correctness Image", cv2.resize((np.multiply(binary_output, label_image) + np.multiply((255 - binary_output), (255 - label_image))) * 255, (512, 512)))

print(np.sum(np.multiply(binary_output, label_image) + np.multiply((255 - binary_output), (255 - label_image))) / shape[0] / shape[1] / 255 / 255)

# Wait for a key press and then close the OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()

Processing: 100%|██████████| 80/80 [00:39<00:00,  2.02it/s]


0.9978790496233215


nan
0
